In [1]:
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

import vbfprocessor
import awkward as ak
import uproot

import os
import time

In [2]:
all_filesets = {
    "VBF_SMEFTsim_topU3l_NP1": [
        "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_NP1/VBF_SMEFTsim_topU3l_NP1_STXS/240425_163141/0000//VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHbox": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHbox/VBF_SMEFTsim_topU3l_Direct_cHbox/230919_233803/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHW5": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHW5/VBF_SMEFTsim_topU3l_Direct_cHW5/240326_201523/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Linear_cHW": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Linear_cHW/VBF_SMEFTsim_topU3l_Linear_cHW/230920_001033/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Quad_cHW": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Quad_cHW/VBF_SMEFTsim_topU3l_Quad_cHW/230920_001106/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Linear_cHWtil": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Linear_cHWtil/VBF_SMEFTsim_topU3l_Linear_cHWtil/230920_003136/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Quad_cHWtil": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Quad_cHWtil/VBF_SMEFTsim_topU3l_Quad_cHWtil/230920_003225/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],   
    "VBF_SMEFTsim_topU3l_Linear_cHbox": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Linear_cHbox/VBF_SMEFTsim_topU3l_Linear_cHbox/230919_235238/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Quad_cHbox": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Quad_cHbox/VBF_SMEFTsim_topU3l_Quad_cHbox/230919_235331/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],  
    "VBF_SMEFTsim_topU3l_Linear_cHj1": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Linear_cHj1/VBF_SMEFTsim_topU3l_Linear_cHj1/230919_233248/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ], 
    "VBF_SMEFTsim_topU3l_Quad_cHj1": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Quad_cHj1/VBF_SMEFTsim_topU3l_Quad_cHj1/230919_233036/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Linear_cHj3": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Linear_cHj3/VBF_SMEFTsim_topU3l_Linear_cHj3/230919_233413/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ], 
    "VBF_SMEFTsim_topU3l_Quad_cHj3": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Quad_cHj3/VBF_SMEFTsim_topU3l_Quad_cHj3/230919_233144/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHj1": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBFH_SMEFTsim_NPall_topU3l_cHj1_1/VBFH_SMEFTsim_NPall_topU3l_cHj1_1/230519_134829/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHj3": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBFH_SMEFTsim_NPall_topU3l_cHj3_1/VBFH_SMEFTsim_NPall_topU3l_cHj3_1/230519_134845/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHW": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBFH_SMEFTsim_NPall_topU3l_cHW_1/VBFH_SMEFTsim_NPall_topU3l_cHW_1/230519_134915/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHWtil": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBFH_SMEFTsim_NPall_topU3l_cHWtil_1/VBFH_SMEFTsim_NPall_topU3l_cHWtil_1/230519_134931/0000//VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_NP1": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_NP1/Hqq_SMEFTsim_topU3l_NP1/230918_201208/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Direct_cHW": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Direct_cHW/Hqq_SMEFTsim_topU3l_Direct_cHW/230920_165636/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Direct_cHWtil": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Direct_cHWtil/Hqq_SMEFTsim_topU3l_Direct_cHWtil/230920_165747/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Direct_cHbox": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Direct_cHbox/Hqq_SMEFTsim_topU3l_Direct_cHbox/230920_165817/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Direct_cHj1": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Direct_cHj1/Hqq_SMEFTsim_topU3l_Direct_cHj1/230920_165843/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Direct_cHj3": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Direct_cHj3/Hqq_SMEFTsim_topU3l_Direct_cHj3/230920_165909/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Linear_cHW": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Linear_cHW/Hqq_SMEFTsim_topU3l_Linear_cHW/230921_161735/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Linear_cHWtil": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Linear_cHWtil/Hqq_SMEFTsim_topU3l_Linear_cHWtil/230921_161938/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Linear_cHbox": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Linear_cHbox/Hqq_SMEFTsim_topU3l_Linear_cHbox/230921_162139/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Linear_cHj1": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Linear_cHj1/Hqq_SMEFTsim_topU3l_Linear_cHj1/230921_162323/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Linear_cHj3": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Linear_cHj3/Hqq_SMEFTsim_topU3l_Linear_cHj3/230921_162449/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Quad_cHW": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Quad_cHW/Hqq_SMEFTsim_topU3l_Quad_cHW/230921_162811/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Quad_cHWtil": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Quad_cHWtil/Hqq_SMEFTsim_topU3l_Quad_cHWtil/230921_164757/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Quad_cHbox": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Quad_cHbox/Hqq_SMEFTsim_topU3l_Quad_cHbox/230921_165005/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Quad_cHj1": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Quad_cHj1/Hqq_SMEFTsim_topU3l_Quad_cHj1/230921_165144/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Quad_cHj3": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Quad_cHj3/Hqq_SMEFTsim_topU3l_Quad_cHj3/230921_165923/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],  
    "VBF_SMEFTsim_topU3l_NP1_start1": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_NP1_start1/VBF_SMEFTsim_topU3l_NP1_start1/240423_191211/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_NP1_Lambda10": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_NP1_Lambda10/VBF_SMEFTsim_topU3l_NP1_Lambda10/240423_191632/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_NP1_noNPprop0": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_NP1_noNPprop0/VBF_SMEFTsim_topU3l_NP1_noNPprop0/240425_163228/0000//VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_NPall1": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_NPall1/VBF_SMEFTsim_topU3l_NPall1/240424_151510/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cdBIm": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cdBIm/VBF_SMEFTsim_topU3l_Direct_cdBIm/240425_225553/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cdBRe": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cdBRe/VBF_SMEFTsim_topU3l_Direct_cdBRe/240425_224546/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cdWIm": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cdWIm/VBF_SMEFTsim_topU3l_Direct_cdWIm/240425_223958/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cdWRe": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cdWRe/VBF_SMEFTsim_topU3l_Direct_cdWRe/240425_223904/0000//VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHB": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHB/VBF_SMEFTsim_topU3l_Direct_cHB/240425_222431/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHBtil": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHBtil/VBF_SMEFTsim_topU3l_Direct_cHBtil/240425_222553/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHd": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHd/VBF_SMEFTsim_topU3l_Direct_cHd/240425_223255/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHDD": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHDD/VBF_SMEFTsim_topU3l_Direct_cHDD/240425_222322/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHu": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHu/VBF_SMEFTsim_topU3l_Direct_cHu/240425_222937/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHudRe": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHudRe/VBF_SMEFTsim_topU3l_Direct_cHudRe/240425_223358/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHudIm": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHudIm/VBF_SMEFTsim_topU3l_Direct_cHudIm/240425_223501/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHWB": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHWB/VBF_SMEFTsim_topU3l_Direct_cHWB/240425_222706/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHWBtil": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHWBtil/VBF_SMEFTsim_topU3l_Direct_cHWBtil/240425_222817/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cuBIm": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cuBIm/VBF_SMEFTsim_topU3l_Direct_cuBIm/240425_224233/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cuBRe": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cuBRe/VBF_SMEFTsim_topU3l_Direct_cuBRe/240425_224112/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cuWIm": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cuWRe/VBF_SMEFTsim_topU3l_Direct_cuWRe/240425_223620/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cuWRe": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cuWIm/VBF_SMEFTsim_topU3l_Direct_cuWIm/240425_223713/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
}

In [3]:
def run_samp(sample):

    fileset = {}
    fileset[sample] = all_filesets[sample]
    outfile = "coffea/"+sample+".coffea"
    
    p = vbfprocessor.VBFProcessor(isMC=True)
    args = {'savemetrics':True, 'schema':NanoAODSchema}

    output = processor.run_uproot_job(
        fileset,
        treename="Events",
        processor_instance=p,
        executor=processor.iterative_executor,#executor=processor.dask_executor,
        executor_args={'schema': NanoAODSchema,'workers': 4},
        chunksize=50000,
        )

    util.save(output, outfile)
    print("saved " + outfile)

In [4]:
for s in all_filesets.keys():
    
    outfile = "coffea/"+s+".coffea"
    if not os.path.exists(outfile) and 'VBF' in s:
        print(s)
        run_samp(s)

VBF_SMEFTsim_topU3l_Direct_cdWIm


Output()

Output()

Output()

saved coffea/VBF_SMEFTsim_topU3l_Direct_cdWIm.coffea
VBF_SMEFTsim_topU3l_Direct_cdWRe


...

Output()

Output()

saved coffea/VBF_SMEFTsim_topU3l_Direct_cuWIm.coffea
VBF_SMEFTsim_topU3l_Direct_cuWRe


Output()

saved coffea/VBF_SMEFTsim_topU3l_Direct_cuWRe.coffea
